In [1]:
!ls

CR_09880W_A_DEF1.fasta	 legend.png		Sc5414_data.txt
DEF1 conversion.xlsx	 plotting.ipynb		Sc5414_data.txt~
DEF1-hp11vw (187).fasta  sc5314 amplicon.fasta	test1.png
hp11vw_data.txt		 sc5314_data.txt~	test.png


This one I am just going to translate the sequences from Jans spreadsheet, just to get the plotting working. I'm sure I have to extract the sequences myself somehow in the future, but not sure at this stage as the seq cordinates seem to be for the reference (I gather I must have these alleles somewhere?)

In [2]:
!cat Sc5414_data.txt

39	GCGCAAGAAAGGGAAGAGTCAAATAAAAGACAACATACA
51	GAATCAAGTGCA
63	GAACCAAATGCA
75	GAACTGAGTACA
87	GAATCAACTACA
99	GAATCAAATGCT
111	GAATCAGGTGCT
123	GAACCAAATGCT
135	GAACCAAGTGCA
147	GAATCAACTACA
159	GAATCAAATGTT
171	GAATCAGGTGCT
183	GAACCAAATGCT
195	GAATCAGGTGCT
207	GAATCAGGTGCT
219	GAACCAACTGCT
231	GAATCAAATGCT
270	GAACTAAAGCAAAGAATTTGGGAAATTTTGAGTTACCGTT


In [3]:
!cat hp11vw_data.txt

39	GCGCAAGAAAGGGAAGAGTCAAATAAAAGACAACATACA
51	GAATCAAGTGCA
63	GAACCAAATGCA
75	GAACTGAGTACA
87	GAATCAACTACA
99	GAATCAAATGCT
111	GAATCAGGTGCT
123	GAATCAGGTGCT
135	GAACCAACTGCT
147	GAATCAAATGCT
186	GAACTAAAGCAAAGAATTTGGGAAATTTTGAGTTACCGT


In [49]:
#%matplotlib inline

In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
# check yeast
from Bio.Alphabet import IUPAC

def collect_repeats(data):
    repeats = []
    for pos, rec in data:
        # just print a warning
        if len(rec) % 3:
            print('not multiple of 3?')
            print("%s-%s" % (pos, rec))
            #repeats.append('XXXXX')
            print(str(rec.translate(table='Alternative Yeast Nuclear')))
        pep = str(rec.translate(table='Alternative Yeast Nuclear'))
        dna = str(rec)
        repeats.append((dna,pep))
    return repeats

def read_rep_data(fname):
    data = []
    with open(fname) as f:
        for line in f:
            bits = line.strip().split('\t')
            pos = bits[0]
            rec = Seq(bits[1], IUPAC.unambiguous_dna)
            data.append((pos, rec))
    return data
       
data1 = read_rep_data('Sc5414_data.txt')    
repeats1 = collect_repeats(data1)
data2 = read_rep_data('hp11vw_data.txt')
repeats2 = collect_repeats(data2)

strains = {'Sc5414':repeats1, 'hp11vw': repeats2}#, 'strain3':repeats3}


not multiple of 3?
270-GAACTAAAGCAAAGAATTTGGGAAATTTTGAGTTACCGTT
ELKQRIWEILSYR


/home/dpidave/miniconda3/lib/python2.7/site-packages/Bio/Seq.py:2423: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


In [2]:
repeats1[0]

('GCGCAAGAAAGGGAAGAGTCAAATAAAAGACAACATACA', 'AQEREESNKRQHT')

In [3]:
import matplotlib.pyplot as plt
import matplotlib.colors
import numpy as np
import seaborn as sns

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(9, 4))

# this was to double up ends to make double wide, doesn't work
#for strain in strains:
#    start = strains[strain][0]
#    end = strains[strain][-1]
#    strains[strain].insert(0, start)
#    strains[strain].insert(-1, end)

# colect all repeats and find unique ones to assign a color
uq_repeats = list(set([repeat[1] for repeat in strains for repeat in strains[repeat]]))

#np.random.seed(sum(map(ord, "palettes")))

col_dict = {}
col_opts = sns.xkcd_rgb.keys()
n = 0
for rep in uq_repeats:
    if rep == 'XXXXX':
        # set bad repeat to black, redundent
        col_dict[rep] = (0,0,0)
    else:
        col_dict[rep] = sns.xkcd_rgb[np.random.choice(col_opts, replace=False)] #palet[n]
    n+=1

## repeat plot
start_end = []
width = 1
spacer = 0

for i,strain in enumerate(strains.keys()):
    used_colors_dict = {}
    used_colors = []
    start = 0
    for dna, repeat in strains[strain]:
        if start == 0:
            # start of seq (not repeat)
            col_dict[repeat] = '#000000'
            col = '#000000'
            start_end.append(repeat)        

        elif repeat == strains[strain][-1][1]:
            # end of seq (not repeat)
            col_dict[repeat] = '#000000'
            col = '#000000'
            start_end.append(repeat)
        else:
            col=col_dict[repeat]
            spacer = 0
        start += width
        if col not in used_colors or col == '#ffffff':
            used_colors.append(col)
            used_colors_dict[col] = [(dna, repeat)]
            hatch=""
        else:
            # pep same, but DNA diff, make hatch
            found,notfound = 0,0
            for d, r in used_colors_dict[col]:
                try:
                    assert r == repeat
                except AssertionError:
                    print('error')
                    print(r)
                    print(repeat)
                if dna != d:
                    notfound+=1
                else:
                    found+=1
            used_colors_dict[col].append((dna, repeat))
            # ocnce it is found, dont hatch it again
            #if notfound != 0 and found ==0:
            # jan seems to hash even if it has been found before
            if notfound != 0:
                hatch="/"
            else:
                hatch = ""
        # could put logic here to make it white box
        axes[0].barh(i,width, left=start - width, color=col_dict[repeat],
                     hatch=hatch, align='center')
    print(used_colors_dict)

# get rid of xtics
axes[0].set_xticklabels('')
ind = np.arange(len(strains))
axes[0].set_yticks(ind)
labels1 = strains.keys()
axes[0].set_yticklabels(labels1)
# remove ticks and borders
axes[0].get_xaxis().set_ticks([])
axes[0].spines['top'].set_visible(False)
axes[0].spines['right'].set_visible(False)
axes[0].spines['bottom'].set_visible(False)
axes[0].spines['left'].set_visible(False)
axes[0].tick_params(axis=u'both', which=u'both',length=0)

# remove the non-repeat start and ends before making the key
assert len(start_end) % 2 == 0
for norep in start_end:
    col_dict.pop(norep, None)
    
# custom key on right plot
col2 = col_dict.values()
labels2 = col_dict.keys()
axes[1].barh(np.arange(len(labels2)),len(labels2)*[0.3], left=0.1, color=col2, align='center')
axes[1].set_xbound((0,1))
axes[1].set_xticklabels('')

# fix ticks
axes[1].set_yticks(np.arange(len(labels2)))
axes[1].set_yticklabels(labels2)
axes[1].get_xaxis().set_ticks([])
plt.tick_params(top='off', bottom='off', left='on', right='off', labelleft='on', labelbottom='on')

# turn of border and ticks
axes[1].spines['top'].set_visible(False)
axes[1].spines['right'].set_visible(False)
axes[1].spines['bottom'].set_visible(False)
axes[1].spines['left'].set_visible(False)
axes[1].tick_params(axis=u'both', which=u'both',length=0)

#frameon=False

fig.tight_layout()
fig.savefig('test.png')
plt.show()

error
AQEREESNKRQHT
ELKQRIWEILSYR
{'#ff6f52': [('GAACCAAGTGCA', 'EPSA')], '#0e87cc': [('GAATCAACTACA', 'ESTT'), ('GAATCAACTACA', 'ESTT')], '#8d5eb7': [('GAACCAAATGCA', 'EPNA'), ('GAACCAAATGCT', 'EPNA'), ('GAACCAAATGCT', 'EPNA')], '#c760ff': [('GAATCAGGTGCT', 'ESGA'), ('GAATCAGGTGCT', 'ESGA'), ('GAATCAGGTGCT', 'ESGA'), ('GAATCAGGTGCT', 'ESGA')], '#c9643b': [('GAATCAAATGCT', 'ESNA'), ('GAATCAAATGCT', 'ESNA')], '#feb308': [('GAATCAAATGTT', 'ESNV')], '#000000': [('GCGCAAGAAAGGGAAGAGTCAAATAAAAGACAACATACA', 'AQEREESNKRQHT'), ('GAACTAAAGCAAAGAATTTGGGAAATTTTGAGTTACCGTT', 'ELKQRIWEILSYR')], '#fcfc81': [('GAACTGAGTACA', 'ESST')], '#2976bb': [('GAACCAACTGCT', 'EPTA')], '#8b3103': [('GAATCAAGTGCA', 'ESSA')]}
error
AQEREESNKRQHT
ELKQRIWEILSYR
{'#0e87cc': [('GAATCAACTACA', 'ESTT')], '#8d5eb7': [('GAACCAAATGCA', 'EPNA')], '#c760ff': [('GAATCAGGTGCT', 'ESGA'), ('GAATCAGGTGCT', 'ESGA')], '#c9643b': [('GAATCAAATGCT', 'ESNA'), ('GAATCAAATGCT', 'ESNA')], '#000000': [('GCGCAAGAAAGGGAAGAGTCAAATAAAAGACAACATA

/home/dpidave/miniconda3/lib/python2.7/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Passing one of 'on', 'true', 'off', 'false' as a boolean is deprecated; use an actual boolean (True/False) instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


<Figure size 900x400 with 2 Axes>

In [4]:
used_colors_dict

{'#000000': [('GCGCAAGAAAGGGAAGAGTCAAATAAAAGACAACATACA', 'AQEREESNKRQHT'),
  ('GAACTAAAGCAAAGAATTTGGGAAATTTTGAGTTACCGT', 'ELKQRIWEILSYR')],
 '#0e87cc': [('GAATCAACTACA', 'ESTT')],
 '#2976bb': [('GAACCAACTGCT', 'EPTA')],
 '#8b3103': [('GAATCAAGTGCA', 'ESSA')],
 '#8d5eb7': [('GAACCAAATGCA', 'EPNA')],
 '#c760ff': [('GAATCAGGTGCT', 'ESGA'), ('GAATCAGGTGCT', 'ESGA')],
 '#c9643b': [('GAATCAAATGCT', 'ESNA'), ('GAATCAAATGCT', 'ESNA')],
 '#fcfc81': [('GAACTGAGTACA', 'ESST')]}

In [5]:
#nice col dict
strains[strain]

[('GCGCAAGAAAGGGAAGAGTCAAATAAAAGACAACATACA', 'AQEREESNKRQHT'),
 ('GAATCAAGTGCA', 'ESSA'),
 ('GAACCAAATGCA', 'EPNA'),
 ('GAACTGAGTACA', 'ESST'),
 ('GAATCAACTACA', 'ESTT'),
 ('GAATCAAATGCT', 'ESNA'),
 ('GAATCAGGTGCT', 'ESGA'),
 ('GAATCAGGTGCT', 'ESGA'),
 ('GAACCAACTGCT', 'EPTA'),
 ('GAATCAAATGCT', 'ESNA'),
 ('GAACTAAAGCAAAGAATTTGGGAAATTTTGAGTTACCGT', 'ELKQRIWEILSYR')]

In [6]:
repeat

'ELKQRIWEILSYR'

In [24]:
a = set([1,2])

In [25]:
for n in a:
    print(n)


1
2


In [2]:
a = {'a':1,'b':2,'c':3}
b = ['a','b']

In [23]:
b.insert(0, 'a')

In [24]:
b

['a', 'a', 'b']